In [26]:
import os
import ssl
import certifi
import urllib3 as urllib
from urllib.request import urlopen
import json
from datetime import datetime
import pandas as pd

In [29]:
ssl._create_default_https_context = ssl._create_unverified_context
# if you are getting an SSL error after running cell 5, it may be necessary to run the above line.
#I dont fully udnerstand how SSL stuff works, so try it without first. The error has to do with the urllib library and security stuff.

In [30]:
def myFunction(forexticker, multiplier, timespan, timefrom, timeto, apikey):
    myurl = "https://api.polygon.io/v2/aggs/ticker/" + forexticker + "/range/" + multiplier + "/" + timespan + "/" + timefrom + "/" + timeto + "?apiKey=" + apikey
    with urlopen(myurl) as response:
        responsebody = response.read()
    return json.loads(responsebody)

In [42]:

multiplier = "1" # the interval of the date range. ie "1" week
timespan = "week" # the interval of the date range. ie 1 "week"
timefrom = "2021-05-19" 
timeto = "2022-05-19"
apikey = "dFPiHNb7JZSCvi61hY_UpPpswbLZapwi"#get your api key from https://api.polygon.io


#create the lists that will populate the pandas table later
date, volume, currency = [],[],[]


In [48]:
# Enter all the currency pairs you want to retrieve from the date range you specified above. Good choices are USDJPY, GBPEUR etc
# Be sure to use the requried format "C:XXXXXX" where the X's are the three letter currency code.
forextickerList = ["C:USDJPY", "C:JPYUSD", "C:USDEUR", "C:EURUSD"]

for pair in forextickerList:
    myjson = myFunction(pair, multiplier, timespan, timefrom, timeto, apikey)
      
    for i in range(len(myjson["results"])):
            #for the length of the list returned by the api, do the loop
        t = int((myjson["results"][i]['t'])/1000 )    
            #datetime is returned in miliseconds which I kept getting errors when converting to a date. 
                #so dividing by 1000 and casting to interger puts it in the correct length and format when using datetime.fromtimestamp
        date.append(datetime.fromtimestamp(t).strftime("%m-%d-%y"))
        volume.append(myjson["results"][i]['v'])
        currency.append(myjson["ticker"])
#print(date, volume, currency)
# ^^ only print this if you want big lists on screen
        


['05-15-21', '05-22-21', '05-29-21', '06-05-21', '06-12-21', '06-19-21', '06-26-21', '07-03-21', '07-10-21', '07-17-21', '07-24-21', '07-31-21', '08-07-21', '08-14-21', '08-21-21', '08-28-21', '09-04-21', '09-11-21', '09-18-21', '09-25-21', '10-02-21', '10-09-21', '10-16-21', '10-23-21', '10-30-21', '11-06-21', '11-13-21', '11-20-21', '11-27-21', '12-04-21', '12-11-21', '12-18-21', '12-25-21', '01-01-22', '01-08-22', '01-15-22', '01-22-22', '01-29-22', '02-05-22', '02-12-22', '02-19-22', '02-26-22', '03-05-22', '03-12-22', '03-19-22', '03-26-22', '04-02-22', '04-09-22', '04-16-22', '04-23-22', '04-30-22', '05-07-22', '05-14-22', '05-15-21', '05-22-21', '05-29-21', '06-05-21', '06-12-21', '06-19-21', '06-26-21', '07-03-21', '07-10-21', '07-17-21', '07-24-21', '07-31-21', '08-07-21', '08-14-21', '08-21-21', '08-28-21', '09-04-21', '09-11-21', '09-18-21', '09-25-21', '10-02-21', '10-09-21', '10-16-21', '10-23-21', '10-30-21', '11-06-21', '11-13-21', '11-20-21', '11-27-21', '12-04-21', '12

In [49]:
#create the data frame and export to csv
myFrame = {
    'date': date,
    'volume': volume,
    'currency': currency
}

polygonIO = pd.DataFrame(myFrame)
print(polygonIO) # get a sneak peak at your data

         date    volume  currency
0    05-15-21  462588.0  C:USDJPY
1    05-22-21  423706.0  C:USDJPY
2    05-29-21  415872.0  C:USDJPY
3    06-05-21  413545.0  C:USDJPY
4    06-12-21  482741.0  C:USDJPY
..        ...       ...       ...
313  04-16-22  776561.0  C:EURUSD
314  04-23-22  925097.0  C:EURUSD
315  04-30-22  914638.0  C:EURUSD
316  05-07-22  966994.0  C:EURUSD
317  05-14-22  862313.0  C:EURUSD

[318 rows x 3 columns]


In [50]:
myfilepath = r'C:\Users\JDavis\Documents'
polygonIO.to_csv(myfilepath+'\\polygondata.csv')